## Description

This example performs a small Monte Carlo simulation, using UQpy's RunModel module and a Matlab model that takes the sum of Normal random variables.  

Michael D. Shields  
December 9, 2018

## Required files

To run this example, the following files must be present in the same directory as this Jupyter script:

1) matlab_model.py  
2) dummy_model.m   
3) process_matlab_output.py   

## Structure of files

1) "matlab_model.py" is the model script. It contains the Python commands to run the Matlab model.     
2) "dummy_model.m" is a template input file is used to create matlab input files (.m) files for each simulation. This text file has placeholders for placement of the sampled variables.   
3) "process_matlab_output.py" is the output script. It is a Python script which processes the output of the Matlab simulations. 

In [1]:
from UQpy.SampleMethods import MCS
from UQpy.RunModel import RunModel
import os
import time

# Add MATLAB to $PATH - modify and use the command in the next line if necessary
# os.system("export PATH=$PATH:'/Applications/MATLAB_R2018a.app/bin'")

## Sample generation

In [2]:
# Call MCS to generate samples
x_mcs = MCS(dist_name=['Normal','Normal','Normal'], dist_params=[[0,1],[0,1],[0,1]], nsamples=3, 
            var_names = ['var1', 'var2', 'var3']) 

print("Three Monte Carlo samples from a trivariate standard normal distribution.")
print(x_mcs.samples)

Three Monte Carlo samples from a trivariate standard normal distribution.
[[-0.18104697 -0.17303674  2.23147144]
 [-0.83662752 -0.00655344  1.21431499]
 [ 0.36246003  2.16429179  0.25836054]]


## Calling RunModel in Serial

In [3]:
# Call to RunModel
t = time.time()
m = RunModel(samples=x_mcs.samples, ntasks=1, model_script='matlab_model.py', input_template='dummy_model.m',
             var_names=x_mcs.var_names, model_object_name="matlab", output_script='process_matlab_output.py',
             output_object_name='read_output', resume=False)
t_ser_matlab = time.time() - t
print("\nTime for serial execution:")
print(t_ser_matlab)

print("The values retured from the Matlab simulation:")
print(m.qoi_list)


Time for serial execution:
19.104127168655396
The values retured from the Matlab simulation:
[array(1.8774), array(0.37113), array(2.7851)]


## Calling RunModel in Parallel
Note that GNU parallel must be installed for this to run.

In [4]:
# Call to RunModel
t = time.time()
m = RunModel(samples=x_mcs.samples, ntasks=4, model_script='matlab_model.py', input_template='dummy_model.m',
             var_names=x_mcs.var_names, model_object_name="matlab", output_script='process_matlab_output.py',
             output_object_name='read_output', resume=False)
t_ser_matlab = time.time() - t
print("\nTime for parallel execution:")
print(t_ser_matlab)

print("The values retured from the Matlab simulation:")
print(m.qoi_list)


Time for parallel execution:
10.67154598236084
The values retured from the Matlab simulation:
[array(2.7851), array(0.37113), array(2.7851)]
